In [ ]:
%matplotlib inline
#%matplotlib notebook

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import os

from ipywidgets import interactive
from ipywidgets import Dropdown, FloatSlider, IntSlider
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import decode_predictions, preprocess_input
from keras.preprocessing import image
from noise_procedural import normalize, perturb, osimplex

# Model & Images

<b>Model:</b> [InceptionV3](https://keras.io/applications/#inceptionv3) NN architecture using Keras weights from training on the ImageNet dataset. This achieves a top-1 accuracy of 77.9% and top-5 accuracy of 93.7%. Other models can be found [here](https://keras.io/applications/#documentation-for-individual-models). 
<br><b>Images:</b> Stock photos taken from [Pexels](https://www.pexels.com/).

In [ ]:
img_dir = 'images/'

# Specify image dimensions
dim_x = 299
dim_y = 299

# Load model
model = InceptionV3(weights = 'imagenet')

# Load images
all_images = {}
for filename in os.listdir(img_dir):
    if not filename.startswith('.'):
        img = image.load_img(img_dir + filename, target_size = (dim_x, dim_y)) # We assume all images have the same dimensions
        img = image.img_to_array(img)
        all_images[filename.split('.')[0]] = img

# Display images
for key, vals in all_images.items():
    fig2 = plt.figure()
    plt.axis('off')
    plt.imshow(vals.astype(np.uint8))

# OpenSimplex Noise

<b>Play</b> with the parameters of the noise function below. You'll find that it's easier to fool the classifier on some images over others!

In [ ]:
# Interactive slider
def PlotOSimplex(img_key, max_norm, period_x, period_y, freq_sine):
    image = all_images[img_key]
    fig = plt.figure(figsize = (18, 8))
    plt.subplots_adjust(wspace = 0.05)
    plt.title('OpenSimplex Noise', size = 16)
    plt.axis('off')

    # Noise pattern
    ax = fig.add_subplot(1, 3, 3)    
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.tick_params(axis = 'both', which = 'both', length = 0)
    ax.set_title('Noise Pattern', size = 14)
    
    noise = osimplex(dim_x = dim_x, dim_y = dim_y, period_x = period_x, period_y = period_y, freq_sine = freq_sine)
    plt.imshow((noise.reshape(dim_x, dim_y, 3) + 1) / 2, cmap = plt.cm.gray)
    
    # Original image
    ax = fig.add_subplot(1, 3, 1)
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.tick_params(axis = 'both', which = 'both', length = 0)
    ax.set_title('Original Image', size = 14)
    payload = perturb(orig = image, max_norm = max_norm, noise = np.zeros((dim_x, dim_y, 3)))
    plt.imshow(payload.astype(np.uint8))
    prob = model.predict(preprocess_input(payload.astype(np.float).reshape((1, dim_x, dim_y, 3))))
    plt.xlabel('\n'.join((item[1]  + '  %.3f' % item[2]) for item in decode_predictions(prob)[0]), horizontalalignment = 'right', x = 1, size = 16)
    
    # Altered image
    ax = fig.add_subplot(1, 3, 2)
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.tick_params(axis = 'both', which = 'both', length = 0)
    ax.set_title('Altered Image', size = 14)
    payload = perturb(orig = image, max_norm = max_norm, noise = noise)
    plt.imshow(payload.astype(np.uint8))
    prob = model.predict(preprocess_input(payload.astype(np.float).reshape((1, dim_x, dim_y, 3))))
    plt.xlabel('\n'.join((item[1]  + '  %.3f' % item[2]) for item in decode_predictions(prob)[0]), horizontalalignment = 'right', x = 1, size = 16)
    
# Parameter sliders
s_img_key = Dropdown(options = list(all_images.keys()), value = 'boat', description = 'Image:')
s_max_norm = IntSlider(min = 0, max = 64, value = 12, step = 2, continuous_update = False, description = 'Max Change:')
s_period_x = IntSlider(min = 2, max = 180, value = 60, step = 1, continuous_update = False, description = 'Period X:')
s_period_y = IntSlider(min = 2, max = 180, value = 60, step = 1, continuous_update = False, description = 'Period Y:')
s_freq_sine = IntSlider(min = 0, max = 60, value = 36, step = 1, continuous_update = False, description = 'Freq Sine:')

interactive(PlotOSimplex, img_key = s_img_key,
                 max_norm = s_max_norm,
                 period_x = s_period_x,
                 period_y = s_period_y,
                 freq_sine = s_freq_sine)

The classifier's top 5 predictions are displayed under each image.